#**Encryption process**



1.   Each client generates its own symmetric key.
2.   The symmetric key is encrypted with the server's public key.
3.   The client encrypts the data with its own symmetric key and sends both the encrypted data and symmetric key to the server.
4.   encrypted data and symmetric key to the server.
The server receives the client's symmetric key using its private key and uses it to decrypt the data.


### In this way:

*   Each client only has the key that can decrpyt its own data stored on local machine
*   Other than the client, the server is the only one that has access to the original symmertical key



In [1]:
import time
import tensorflow as tf
from cryptography.fernet import Fernet
import numpy as np

In [2]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

In [3]:
!pip install --upgrade pycryptodome

In [4]:
def generate_key_pair():
    server_key = RSA.generate(2048)
    with open('server_private_key.pem', 'wb') as f:
        f.write(server_key.export_key())

    # Save server's public key (for encryption)
    with open('server_public_key.pem', 'wb') as f:
        f.write(server_key.publickey().export_key())

    return server_key


In [5]:
server_key = generate_key_pair()

In [6]:
# Create a TensorFlow tensor with shape (1, 28, 28, 512)
data_tensor = tf.random.normal((1, 28, 28, 512))

# Convert tensor data to bytes
data_bytes = tf.io.serialize_tensor(data_tensor).numpy()

# **Client**

In [7]:
# Generate a random key (should be kept secret and shared securely)
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Encrypt symmetric key using server's public key
with open('server_public_key.pem', 'rb') as f:
    server_public_key = RSA.import_key(f.read())
cipher_rsa = PKCS1_OAEP.new(server_public_key)
encrypted_symmetric_key = cipher_rsa.encrypt(key)

# Initialize Fernet cipher with the generated key
cipher_suite = Fernet(key)

In [8]:
def encrypt_embeddings(data, cipher_suite):
    start_time = time.time()
    encrypted_data = cipher_suite.encrypt(data)
    end_time = time.time()
    return encrypted_data, end_time - start_time

In [9]:

encrypted_tensor, encryption_time = encrypt_embeddings(data_bytes, cipher_suite)

# **Server**

In [10]:
# Decrypt client's symmetric key
cipher_rsa = PKCS1_OAEP.new(server_key)
decrypted_client_symmetric_key = cipher_rsa.decrypt(encrypted_symmetric_key)

# Decrypt data using symmetric key
cipher_suite = Fernet(decrypted_client_symmetric_key)


In [11]:
def decrypt_embeddings(encrypted_data, cipher_suite):
    start_time = time.time()
    decrypted_data = cipher_suite.decrypt(encrypted_data)
    end_time = time.time()
    tensor = tf.io.parse_tensor(decrypted_data, out_type=tf.float32)
    return tensor, end_time - start_time

In [13]:
tensor, decryption_time = decrypt_embeddings(encrypted_tensor, cipher_suite)

In [14]:
tensor == data_tensor

<tf.Tensor: shape=(1, 28, 28, 512), dtype=bool, numpy=
array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True